# Introduction

- Introduction to the perturbation methods (state and step-size perturbation).
- Create Izhikevich example.

In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import matplotlib.patches as patches
from scipy.stats import norm, uniform

In [ ]:
from sys import path as sys_path
from os.path import abspath as os_path_abspath
sys_path.append(os_path_abspath('..'))
import addpaths

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import data_utils
import plot_utils as pltu
import ode_solver

# Toy problem

In [ ]:
import test_models
toy_model = test_models.model1()

In [ ]:
plt.plot(np.linspace(0,3,101), toy_model.eval_y(t=np.linspace(0,3,101)));

## Plot

### Plot parameters

In [ ]:
stages_kw  = {'lw': 0.8, "alpha": 0.7, "ls": '--', "zorder": -20, "c": 'gray'}
exact_kw   = {'lw': 0.8, "alpha": 0.7, "ls": '-',  "zorder": -10}
sol_kw     = {"alpha": 1.0, "ms": 4.5, "markeredgewidth": 0}
error_kw   = {"alpha": 0.5, "color": 'red', "lw": 1.2, "zorder": -3}
arrowstyle = "Simple,tail_width=1.0,head_width=3,head_length=8"
arrowcolor = 'blue'
dist_kw    = {'lw': 2, "alpha": 0.4, "zorder": -30, "c": 'green'}

### Conrad's Method

In [ ]:
def plot_conrads_method(model, ax=None):
    """Plot conrads perturbation method"""
    
    if ax is None: ax = plt.subplot(111)
    
    t0 = 2.5
    y0 = model.eval_y(t0)
    ydot0 = model.eval_ydot(t0, y0)

    h = 0.4
    t1 = t0 + h

    true_ts = np.linspace(t0,t0+2*h,101)
    true_ys = model.eval_y(true_ts)

    y1_FE = y0 + h*ydot0 # Forward Euler
    ydot1_FE = model.eval_ydot(t1, y1_FE)
    y1_HN = y0 + h*(0.5*ydot0 + 0.4*ydot1_FE) # Heun's method

    dt_ydot = 0.55*h

    xlims = [t0-0.05*h, t1+dt_ydot+0.05*h]
    ylims = [y0-0.15*np.abs(y0-y1_FE), max(y1_FE, y1_HN)+0.45*np.abs(y0-y1_FE)]
    ax.set(xlim=xlims, ylim=ylims)

    ax.plot(true_ts, true_ys, c='k', label=r"exact", **exact_kw)

    ax.plot(t0, y0, 'o', c=arrowcolor, **sol_kw)
    ax.plot([xlims[0], t0, t0], [y0, y0, ylims[0]], **stages_kw)

    ax.plot(t1, y1_FE, marker="o", color= 'darkred', **sol_kw)
    ax.plot(t1, y1_HN, marker="X", color= 'darkred', **sol_kw)
    ax.vlines(t1, y1_FE, y1_HN, **error_kw)

    ax.plot([xlims[0], t1], [y1_FE, y1_FE], **stages_kw)
    ax.plot([xlims[0], t1], [y1_HN, y1_HN], **stages_kw)
    ax.plot([t1, t1], [max(y1_FE, y1_HN), ylims[0]], **stages_kw)

    for i, (ti, yi, ydoti) in enumerate([(t0, y0, ydot0), (t1, y1_FE, ydot1_FE)]):

        arrow = patches.FancyArrowPatch(
            (ti, yi), (ti+dt_ydot, yi+dt_ydot*ydoti),
            color=arrowcolor, alpha=.8, lw=0, arrowstyle=arrowstyle, clip_on=False
        )
        ax.add_patch(arrow)

        ax.text(ti+0.5*dt_ydot, yi+max(0.48*dt_ydot*ydoti, 0.52*dt_ydot*ydoti),
                r'$k_' + str(i) + '$', ha='right', va='bottom', c=arrowcolor, clip_on=False) 

    ax.set_xticks([t0, t1])
    ax.set_xticklabels([r'$t_0$', r'$t_1 = t_0 + \Delta t$'], fontsize=plt.rcParams['font.size'])

    ax.set_yticks([y0, y1_FE, y1_HN])
    ax.set_yticklabels([r'$x_0$', r'${x}^\mathregular{FE}_1$', r'${x}^\mathregular{HN}_1$'],
                       fontsize=plt.rcParams['font.size'])

    y_pdf_in = np.linspace(ylims[0], ylims[1], 101)
    y_pdf_out = norm(loc=y1_FE, scale=abs(y1_FE-y1_HN)).pdf(y_pdf_in)
    y_pdf_out /= np.max(y_pdf_out)
    y_pdf_out *= 0.2*h
    y_pdf_out += t1
    ax.plot(y_pdf_out, y_pdf_in, **dist_kw)

    y_pdf_idx = 30
    ax.annotate(
        r'$\mathregular{pdf}({x}_1$)', xy=(np.max(y_pdf_out), y_pdf_in[np.argmax(y_pdf_out)]),
        xytext=(np.max(y_pdf_out)+0.2*h, y_pdf_in[y_pdf_idx]), c=dist_kw['c'],
        arrowprops=dict(facecolor=dist_kw['c'], shrink=0.05, width=2, headwidth=8, alpha=dist_kw['alpha']),
        ha='center', va='top'
    )


In [ ]:
fig, ax = pltu.subplots(1,1, xsize=4)
plot_conrads_method(ax=ax, model=toy_model)

### Abdulle's method

In [ ]:
def plot_abdulles_method(model, ax=None):

    if ax is None: ax = plt.subplot(111)
    
    t0 = 2.5
    y0 = model.eval_y(t0)
    ydot0 = model.eval_ydot(t0, y0)

    h = 0.4
    t1 = t0 + h
    
    tau = 0.75
    dt_pert = tau*h**1.5 # Bound of uniform distribution
    t1a = t1-dt_pert
    t1b = t1+dt_pert

    true_ts = np.linspace(t0,t0+2*h,101)
    true_ys = model.eval_y(true_ts)

    t1_smp = 0.6*t1a + 0.4*t1
    h_smp = t1_smp-t0
    

    y1_FE = y0 + h*ydot0 # Forward Euler
    y1_FE_pert = y0 + h_smp*ydot0 # Forward Euler

    dt_ydot = 0.99*h

    xlims = [t0-0.05*h, t1b+0.25*h]
    ylims = [y0-0.2*np.abs(y0-y1_FE), y1_FE+0.55*np.abs(y0-y1_FE)]

    ax.set(xlim=xlims, ylim=ylims)

    ax.plot(true_ts, true_ys, c='k', label=r"exact", **exact_kw)

    ax.plot(t0, y0, 'o', c=arrowcolor, **sol_kw)
    ax.plot([xlims[0], t0, t0], [y0, y0, ylims[0]], **stages_kw)

    ax.plot(t1, y1_FE, marker="o", color= 'darkred', **sol_kw)
    ax.plot(t1_smp, y1_FE_pert, marker="X", color='darkred', **sol_kw)
    ax.plot(t1, y1_FE_pert, marker="o", color=dist_kw['c'], **sol_kw)

    ax.plot([xlims[0], t1], [y1_FE, y1_FE], **stages_kw)
    ax.plot([xlims[0], t1], [y1_FE_pert, y1_FE_pert], **stages_kw)
    ax.plot([t1, t1], [max(y1_FE, y1_FE_pert), ylims[0]], **stages_kw)
    ax.plot([t1_smp, t1_smp], [ylims[0], y1_FE_pert], **stages_kw)

    for i, (ti, yi, ydoti) in enumerate([(t0, y0, ydot0)]):

        arrow = patches.FancyArrowPatch(
            (ti, yi), (ti+dt_ydot, yi+dt_ydot*ydoti),
            color=arrowcolor, alpha=.8, lw=0, arrowstyle=arrowstyle, clip_on=False
        )
        ax.add_patch(arrow)

        ax.text(ti+0.38*dt_ydot, yi+max(0.35*dt_ydot*ydoti, 0.42*dt_ydot*ydoti),
                r'$k_' + str(i) + '$', ha='right', va='bottom', c=arrowcolor, clip_on=False)


    ax.set_xticks([t0, t1a, t1, t1b, t1_smp])
    ax.set_xticklabels([r'$t_0$', r'$t_1 - a$', r'$t_1 = t_0 + \Delta t$', r'$t_1 + a$', r'$t_1^*$'],
                       fontsize=plt.rcParams['font.size'])

    ax.set_yticks([y0, y1_FE, y1_FE_pert])
    ax.set_yticklabels([r'$x_0$', r'${x}^\mathregular{FE}_1$', r'${x}_1^*$'],
                       fontsize=plt.rcParams['font.size'])

    t_pdf_in = np.linspace(xlims[0], xlims[1], 1001)
    t_pdf_out = uniform(loc=t1a, scale=t1b-t1a).pdf(t_pdf_in)
    t_pdf_out /= np.max(t_pdf_out) / (0.2*(ylims[1]-ylims[0]))
    t_pdf_out += np.mean([y0, ylims[0]])
    ax.plot(t_pdf_in, t_pdf_out, **dist_kw, ls='dashdot')

    ax.annotate(
        r'$\mathregular{pdf}(t_1^\mathregular{eval})$',
        xy=(t1b-0.2*h, max(t_pdf_out)),
        xytext=(t1b+0.1*h, max(t_pdf_out)+0.11*(ylims[1]-ylims[0])), c=dist_kw['c'],
        arrowprops=dict(facecolor=dist_kw['c'], shrink=0.05, width=2, headwidth=8, alpha=dist_kw['alpha']),
        ha='center', va='bottom'
    )

    y_pdf_in = np.linspace(ylims[0], ylims[1], 1001)
    y_pdf_out = uniform(loc=y0+(t1a-t0)*ydot0, scale=(t1b-t1a)*ydot0).pdf(y_pdf_in)
    y_pdf_out /= np.max(y_pdf_out) / (0.1*(xlims[1]-xlims[0]))
    y_pdf_out += t1
    ax.plot(y_pdf_out, y_pdf_in, **dist_kw)

    ax.annotate(
        r'$\mathregular{pdf}({x}_1)$',
        xy=(max(y_pdf_out), y1_FE),
        xytext=(max(y_pdf_out)+0.3*h, np.mean([y1_FE, ylims[1]])),
        c=dist_kw['c'], ha='center', va='bottom',
        arrowprops=dict(facecolor=dist_kw['c'], shrink=0.05, width=2, headwidth=8, alpha=dist_kw['alpha']),
    )

In [ ]:
fig, ax = pltu.subplots(1,1, xsize=5)
plot_abdulles_method(ax=ax, model=toy_model)

# A real model, the Izhikevich neuron

In [ ]:
import izhikevich
import izhikevich_parameters
from data_generator_IN import data_generator_IN

parameters = izhikevich_parameters.parameters()
neuron_parameters, stimulus_parameters, t_parameters = parameters.select_mode('Tonic spiking')

IN_model = izhikevich.neuron(neuron_parameters, stimulus_parameters)

In [ ]:
gen = data_generator_IN(
    y0=IN_model.y0, t0=11, h0=0.1, tmax=15,
    gen_acc_sols=True, return_vars=['ys', 'events'],
    model=IN_model, n_samples=40, base_folder='_data'
)
gen.DEBUG = True

In [ ]:
# Deterministic solution
soldet = gen.gen_det_sol(method='FE', adaptive=0, step_param=t_parameters['dt'], plot=True)

In [ ]:
# Probabilistic solution
solprob = gen.gen_sol(method='FE', adaptive=0, pert_method='conrad', step_param=t_parameters['dt'], plot=True)

## Plot

In [ ]:
def plot_IN_traces(ax):
    """Plot traces of Izhikevich neuron"""
    example_kws = {'lw': 0.8, "alpha": 0.7, "marker": '.', "ms": 3.5, "ls": '-'}
    determi_kws = {'lw': 0.8, "alpha": 0.7, "marker": '.', "ms": 3.5, "ls": '--'}
    
    ax.plot(solprob.ts-gen.t0, solprob.get_ys(yidx=0, sampleidx=1), **example_kws, clip_on=False)
    ax.plot(solprob.ts-gen.t0, solprob.get_ys(yidx=0, sampleidx=5), **example_kws, clip_on=False)
    ax.plot(solprob.ts-gen.t0, soldet.get_ys(yidx=0), c='black', **determi_kws, clip_on=False)
    
    # Annotate and 30 mV line.
    ax.axhline(30, c='gray', ls=':', lw=0.8)
    ax.set_ylabel('v(t)')
    ax.set_xlabel('Time (ms)')
    ax.set_ylim(-75, 75)

In [ ]:
plot_IN_traces(plt.subplot(111))

# Figure

In [ ]:
fig, axs = pltu.subplots(1,3,xsize='text',gridspec_kw=dict(height_ratios=[1,1,0.6]),ysizerow=1.)

axs = axs.flatten()

plot_conrads_method(model=toy_model, ax=axs[0])
plot_abdulles_method(model=toy_model, ax=axs[1])
plot_IN_traces(axs[2])

pltu.move_xaxis_outward(axs[2], scale=3)
pltu.make_share_xlims(axs[:2])

pltu.tight_layout(h_pad=0.6, rect=[0,0,1,0.98])
sns.despine()

for panel_num, ax in zip('ABCDEF', axs):
    ax.text(-0.09, 1, panel_num+" "*10, ha='left', va='center', fontweight='bold',
            fontsize=plt.rcParams['axes.titlesize'], transform=ax.transAxes)

pltu.savefig('introduction_both_pert_methods')
plt.show()

pltu.show_saved_figure(fig)